In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# Import Library

In [2]:
# sub package download
!pip install -q mxnet
!pip install -q gluonnlp pandas tqdm
!pip install -q sentencepiece
!pip install -q torch
!pip install -q transformers==3

In [3]:
# model structure download
!pip install -q git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
# os path
import os

# data analysis
import pandas as pd

# nlp
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
BASE_DIR = "/gdrive/My Drive/datascience/goverment_hackerton/emotion_analysis" # 로컬 경로로 바꿔주어야 로컬에서 작동됨

CODE_DIR = os.path.join(BASE_DIR, "code")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODEL_DIR = os.path.join(BASE_DIR, "model")

DATA_BINARY_DIR = os.path.join(DATA_DIR, "binary_label")
DATA_MULTI_DIR = os.path.join(DATA_DIR, "multi_label")

TRAIN_DATA_DIR = os.path.join(DATA_MULTI_DIR, "translated_train.csv")
TEST_DATA_DIR = os.path.join(DATA_MULTI_DIR, "translated_test.csv")

# Model Configuration Setting

In [18]:
# GPU 사용 시
# device = torch.device("cuda:0")
# CPU 사용 시
device = torch.device("cpu")

In [19]:
# model load
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [20]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [21]:
emotion2label = {"neutral" : 0, "sadness" : 1, "fear" : 2, "anger" : 3, "joy" : 4}
label2emotion = { 0 : "neutral", 1 : "sadness",  2 : "fear",  3 : "anger",  4 : "joy"}

In [22]:
## Setting parameters
max_len = 64 ############################## 문항 개수가 64개가 넘어가지 않으면 한번에 처리함
batch_size = 64
""" Training configuration
warmup_ratio = 0.1
num_epochs = 5 
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
"""

' Training configuration\nwarmup_ratio = 0.1\nnum_epochs = 5 \nmax_grad_norm = 1\nlog_interval = 200\nlearning_rate =  5e-5\n'

In [23]:
class BERTDatasetForTest(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        texts = dataset["sentence"].tolist()
        # labels = dataset["Emotion"].tolist()

        self.sentences = [transform([text]) for text in texts]
        # self.labels = [np.int32(emotion2label[label]) for label in labels]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))


In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Get Test Data

In [25]:
real_test_data = pd.read_csv(os.path.join(DATA_MULTI_DIR, "pos_neg_test.txt"))
real_test_data.head(10)

,sentence
0,오랜만에 만나서 반갑다는 얘기를 할 것 같다.
1,나에 대해 관심이 없어서 얘기할 것 같지 않다.
2,나를 동정할 것 같다. 최근에 안좋은 일이 많았기 때문에..
3,친구랑 놀고 다시 혼자가 되면 머릿속이 복잡해진다.
4,역시 사람을 만나는건 너무 힘들어
5,오늘 친구에게 실수한 것이 없는지 곱씹어보고 괜시리 불안해함
6,나는 너무 부주의해. 매일 실수만 한다.
7,애인이 나에 대해 실망할 것 같아서 슬퍼진다.
8,애인이 알기 전에 새로 사놓아야 겠다
9,그래도 이땐 행복했었구나 하는 생각. 기분이 묘하다


In [26]:
data_real_test = BERTDatasetForTest(real_test_data, tok, max_len, True, False)

In [27]:
real_test_dataloader = torch.utils.data.DataLoader(data_real_test, batch_size=batch_size, num_workers=5)

# Get Pretrained Model Parameter

In [29]:
PATH = os.path.join(MODEL_DIR, os.path.join(MODEL_DIR,"multi_label_model.pt"))
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))


<All keys matched successfully>

# Run Inference

In [30]:
model.eval()
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(real_test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    #label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


# Output Check

In [31]:
max_vals, max_indices = torch.max(out, 1)

In [32]:
predicted_emotion = list(label2emotion[label] for label in max_indices.tolist())

In [33]:
inference = pd.DataFrame({"Text" : real_test_data["sentence"].tolist(), "Emotion" : predicted_emotion})
inference

,Text,Emotion
0,오랜만에 만나서 반갑다는 얘기를 할 것 같다.,joy
1,나에 대해 관심이 없어서 얘기할 것 같지 않다.,sadness
2,나를 동정할 것 같다. 최근에 안좋은 일이 많았기 때문에..,sadness
3,친구랑 놀고 다시 혼자가 되면 머릿속이 복잡해진다.,sadness
4,역시 사람을 만나는건 너무 힘들어,sadness
5,오늘 친구에게 실수한 것이 없는지 곱씹어보고 괜시리 불안해함,fear
6,나는 너무 부주의해. 매일 실수만 한다.,sadness
7,애인이 나에 대해 실망할 것 같아서 슬퍼진다.,sadness
8,애인이 알기 전에 새로 사놓아야 겠다,fear
9,그래도 이땐 행복했었구나 하는 생각. 기분이 묘하다,joy
